# Installing Dependencies

In [1]:
pip install lyricsgenius

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 kB 2.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [23]:
from striprtf.striprtf import rtf_to_text
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch
import numpy as np
import pandas as pd
from datetime import datetime
from dateutil import parser
import matplotlib.pyplot as plt
import json
import lyricsgenius

# Reading Lyrics (.rtf) File and Converting to String

In [8]:
with open("When I Was Your Man.rtf") as file:
    content = file.read()
    text = rtf_to_text(content)
print(text)

"When I Was Your Man"

Same bed but it feels just a little bit bigger now
Our song on the radio but it don't sound the same
When our friends talk about you, all it does is just tear me down
'Cause my heart breaks a little when I hear your name

It all just sounds like oooooh…
Mmm, too young, too dumb to realize
That I should've bought you flowers
And held your hand
Should've gave you all my hours
When I had the chance
Take you to every party
'Cause all you wanted to do was dance
Now my baby's dancing
But she's dancing with another man

My pride, my ego, my needs, and my selfish ways
Caused a good strong woman like you to walk out my life
Now I'll never, never get to clean up the mess I made, ohh…
And it haunts me every time I close my eyes

It all just sounds like oooooh…
Mmm, too young, too dumb to realize
That I should've bought you flowers
And held your hand
Should've gave you all my hours
When I had the chance
Take you to every party
'Cause all you wanted to do was dance
Now my bab

# Using Genius API to Import Lyrics (main resource: https://github.com/johnwmillr/LyricsGenius#usage)

In [7]:
import os
import getpass

In [8]:
os.environ["GENIUS_API_KEY"] = getpass.getpass()
genius = lyricsgenius.Genius(os.environ.get("GENIUS_API_KEY"))

In [9]:
album = genius.search_album("Unorthodox Jukebox", "Bruno Mars")
album.save_lyrics()
# a .json file is saved in the directory containing information for the whole album

Searching for "Unorthodox Jukebox" by Bruno Mars...
Skipping file save.



In [10]:
with open("Lyrics_UnorthodoxJukebox.json") as f:
    lyrics = json.load(f)

In [11]:
# lyrics now stores the .json file as a python string, contains a lot of different fields, key ones for our purposes
# are 'tracks', and within 'tracks', 'song'
print(lyrics['tracks'][3]['number'])

4


In [20]:
# To store our lyrics, we'll be using python dictionaries with two keys each to store the song title and their lyrics
# We use string splicing to deal with ignoring some extra meta-info included within the 'lyrics' field

def lyric_database(lyrics):
    dict = {"song": [], "lyrics": []}

    for track in lyrics['tracks']:
        new_lyrics = track['song']['lyrics']
        start_index = new_lyrics.find('[Verse 1]')
        new_lyrics = new_lyrics[start_index:]
        
        end_index = new_lyrics.find('Embed')
        new_lyrics = new_lyrics[:(end_index-2)]
        
        dict["song"].append(track['song']['title_with_featured'])
        dict["lyrics"].append(new_lyrics)

    return dict

In [21]:
song_lyrics_dict = {"song": [], "lyrics": []}
song_lyrics_dict = lyric_database(lyrics)

In [22]:
# Example of the info stored within our python dictionary
print(song_lyrics_dict["song"][4])
print("-------")
print(song_lyrics_dict["lyrics"][4][:50])

Moonshine
-------
[Verse 1]
Hello
You know, you look even better tha


# Sentiment Analyzer Model
## Trying out Lyric Genre Model from huggingface(https://huggingface.co/Veucci/lyric-to-genre?text=When+I%27m+away+from+you%2C+I%27m+happier+than+ever+Wish+I+could+explain+it+better+I+wish+it+wasn%27t+true+Give+me+a+day+or+two+to+think+of+something+clever+To+write+myself+a+letter+To+tell+me+what+to+do%2C+mm-mmm+Do+you+read+my+interviews%3F+Or+do+you+skip+my+avenue%3F+%28My+avenue%29+When+you+%28when+you%29+said+you+were+passing+through+Was+I+even+on+your+way%3F+I+knew+when+I+asked+you+to+%28when+I+asked+you+to%29+Be+cool+about+what+I+was+telling+you+You%27d+do+the+opposite+of+what+you+said+you%27d+do+%28what+you+said+you%27d+do%29+And+I%27d+end+up+more+afraid+Don%27t+say+it+isn%27t+fair+You+clearly+weren%27t+aware+that+you+made+me+miserable+So+if+you+really+wanna+know)

In [24]:
classifier = pipeline("text-classification", model="Veucci/lyrics-to-genre")

In [33]:
result = classifier(song_lyrics_dict["lyrics"][9])
print(result)

[{'label': 'Pop', 'score': 0.999990701675415}]


In [32]:
print(song_lyrics_dict["lyrics"][9][:50])

[Verse 1]
Oh, oh, oh, I
I was a city boy
Right int


In [34]:
# Works, obviously all songs in the album have a dominantly pop genre rating